In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import pandas as pd


In [2]:
checkpoint = "unsloth/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model.config.model_max_length = 2048

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [2]:
review_data = pd.read_csv("/content/drive/MyDrive/sem6-project/datasets/CombinedReviewDataset.csv")
review_data

,productName,review
0,SBL Silk' n Stay Aloevera Cream 100g each [Pac...,Oily.
1,SBL Silk' n Stay Aloevera Cream 100g each [Pac...,Nyce
2,SBL Silk' n Stay Aloevera Cream 100g each [Pac...,Very good
3,SBL Silk' n Stay Aloevera Cream 100g each [Pac...,Very good
4,SBL Silk' n Stay Aloevera Cream 100g each [Pac...,Osum cream bt not for whiting
...,...,...
27918,OPPO K12x 5G with 45W SUPERVOOC Charger In-The...,Very nyccc..Amazing
27919,OPPO K12x 5G with 45W SUPERVOOC Charger In-The...,Camera is avarage battery is good display is n...
27920,OPPO K12x 5G with 45W SUPERVOOC Charger In-The...,Ok
27921,OPPO K12x 5G with 45W SUPERVOOC Charger In-The...,Phone Nicely


In [3]:
review_data.shape

(27923, 2)

In [4]:
# type(review_data.iloc[0])
review_data.iloc[0].iloc[1]

'Oily.'

In [ ]:
review_dict = {}
for i in range(review_data.shape[0]):
  key = review_data.iloc[i].iloc[0]  # Get the product name
  value = review_data.iloc[i].iloc[1]  # Get the review text
  if key in review_dict:
    # If the key exists, append the new review text
    review_dict[key].append(value)
  else:
    # If the key doesn't exist, create a new entry
    review_dict[key] = [value]
review_dict

In [5]:
prompt = [{"role": "System",
           "content": "Given this concatenated review, seperated by '||', summarize it as pros and cons of product (if paussible)."},
          {
              "role": "User",
              "content": "Review: "
          }]
def generate_prompt(reviews):
  return f"""You are a product review summarizer. Given this concatenated reviews, seperated by '||', summarize it as list of 'pros' and 'cons' of product under 10 lines.

Reviews: {reviews}
"""

In [17]:
def summarize_review(review):
  generated_prompt = generate_prompt(review)
  print(generated_prompt)

  tokenizer.pad_token = tokenizer.eos_token
  input = tokenizer(generated_prompt, return_tensors="pt")
  # print("Input is \n", input)
  print("Input is tokenized\n")

  output = model.generate(**input)
  # print(output)
  print("output is generated")

  summarized_review = tokenizer.batch_decode(output, skip_special_tokens=True)
  print(summarized_review)
  print("\nnext operation")
  return summarized_review

In [18]:
summary_dict = {}
count=0
max_length = 500
for k, v in review_dict.items():
  if count==1:
    break
  # print(k, v)
  print("no of reviews for this key: ", len(v))
  review=""
  for reviews in v:
    if len(review)<max_length:
      review += ' || ' + reviews
    else:
      summarized_review = summarize_review(review)
      # print(summarized_review)
      review = reviews

  count+=1

no of reviews for this key:  100
You are a product review summarizer. Given this concatenated reviews, seperated by '||', summarize it as list of 'pros' and 'cons' of product under 10 lines.

Reviews:  || Oily. || Nyce || Very good || Very good || Osum cream bt not for whiting || Baje cream don't waist Manny || Amazing || Nice || Good product || It's a nice || Nice products and first delivery i am happy this product thank you Flipkart ❤️ || Nice product || Good || Nice product. || Nice product || Nice || Very nice product || Thanks to flipkart || Thank you Flipcart for this wonderful product this cream is really very amazing..... || Itz written pack of 2 hut I get only 1 very disturbing || Nice product

Input is tokenized



KeyboardInterrupt: 

In [ ]:
summary_dict

# using facebook bart summarizzer model

In [6]:

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [12]:
def summarize(reviews):
  max_length = 512 # Adjust as needed based on your model's limitations
  reviews = reviews[:max_length]

  # 2. Handle unknown tokens (optional)
  summarizer.tokenizer.add_special_tokens({'additional_special_tokens': ['<UNK>']})
  prompt = f"Summarize this concatenated review as list of pros and cons. Reviews: {reviews}"
  return summarizer(reviews, max_length=130, min_length=30, do_sample=False)

In [8]:
print(summarize(ARTICLE))

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [9]:
review_dict_string = {}
for i in range(review_data.shape[0]):
  key = review_data.iloc[i].iloc[0]  # Get the product name
  value = review_data.iloc[i].iloc[1]  # Get the review text
  if key in review_dict_string:
    # If the key exists, append the new review text
    review_dict_string[key] += ", "+ value
  else:
    # If the key doesn't exist, create a new entry
    review_dict_string[key] = value

review_dict_string

Output hidden; open in https://colab.research.google.com to view.

In [13]:
len(review_dict_string)

310

In [15]:
summary_dict = {}
count=0
max_length = 500
for k, v in review_dict_string.items():
  # print(k,"\n", v)
  if k in summary_dict:
    print("already found summary")
    count+=1
    continue
  output = summarize(v)
  print(f"summarized output is generated: {count} of {len(review_dict_string)} \n")
  summary_dict[k] = output
  count+=1


summarized output is generated: 0 of 310 

summarized output is generated: 1 of 310 



Your max_length is set to 130, but your input_length is only 126. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)


summarized output is generated: 2 of 310 

summarized output is generated: 3 of 310 

summarized output is generated: 4 of 310 

summarized output is generated: 5 of 310 

summarized output is generated: 6 of 310 

summarized output is generated: 7 of 310 

summarized output is generated: 8 of 310 

summarized output is generated: 9 of 310 



Your max_length is set to 130, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


summarized output is generated: 10 of 310 



...

summarized output is generated: 303 of 310 

summarized output is generated: 304 of 310 



Your max_length is set to 130, but your input_length is only 129. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


summarized output is generated: 305 of 310 

summarized output is generated: 306 of 310 

summarized output is generated: 307 of 310 



Your max_length is set to 130, but your input_length is only 125. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)


summarized output is generated: 308 of 310 

summarized output is generated: 309 of 310 



In [16]:
summary_dict

{"SBL Silk' n Stay Aloevera Cream 100g each [Pack of 2 ] [100gX2 , 200 g]\xa0\xa0(200 g)": [{'summary_text': "Oily., Nyce, very good, Very good, Osum cream bt not for whiting, Baje cream don't waist Manny, Amazing, Nice, Good product, It's a nice, Nice products."}],
 "LORD'S CREAM FOR COMPLEXION PACK OF 2 (EACH OF 25 GM )\xa0\xa0(50 g)": [{'summary_text': 'Good, Thanks, Ok, Good product, Good, Nice, Worst product, Super product must buy this..., Very nice, good product, Low quality product...., Very good ptoduct.'}],
 'Deyga Aloe Vera Gel 100% Pure Multipurpose for Face, Skin & Hair\xa0\xa0(100 g)': [{'summary_text': 'Good product, Very effective, Using it .. I like it, Nice, Good, The best aloevera gel i ever used., Nice, Alovera is pure But price is very high Please deyga make it affordable.'}],
 'Shahnaz Husain SHAMOIST 50GM\xa0\xa0(50 ml)': [{'summary_text': 'Good but darker in face after 1 week use, good, good product. Using for last 25years, Good mast, Very bad product, I like it

In [18]:
df = pd.DataFrame(summary_dict).T

In [19]:
df.to_csv("/content/drive/MyDrive/sem6-project/datasets/review_summary_dict.csv")

In [20]:
newdf = pd.read_csv("/content/drive/MyDrive/sem6-project/datasets/review_summary_dict.csv")

In [54]:
x = newdf.T.to_dict('list')
x

{0: ["SBL Silk' n Stay Aloevera Cream 100g each [Pack of 2 ] [100gX2 , 200 g]\xa0\xa0(200 g)",
  '{\'summary_text\': "Oily., Nyce, very good, Very good, Osum cream bt not for whiting, Baje cream don\'t waist Manny, Amazing, Nice, Good product, It\'s a nice, Nice products."}'],
 1: ["LORD'S CREAM FOR COMPLEXION PACK OF 2 (EACH OF 25 GM )\xa0\xa0(50 g)",
  "{'summary_text': 'Good, Thanks, Ok, Good product, Good, Nice, Worst product, Super product must buy this..., Very nice, good product, Low quality product...., Very good ptoduct.'}"],
 2: ['Deyga Aloe Vera Gel 100% Pure Multipurpose for Face, Skin & Hair\xa0\xa0(100 g)',
  "{'summary_text': 'Good product, Very effective, Using it .. I like it, Nice, Good, The best aloevera gel i ever used., Nice, Alovera is pure But price is very high Please deyga make it affordable.'}"],
 3: ['Shahnaz Husain SHAMOIST 50GM\xa0\xa0(50 ml)',
  "{'summary_text': 'Good but darker in face after 1 week use, good, good product. Using for last 25years, Good ma

In [62]:
a = pd.read_csv("/content/drive/MyDrive/sem6-project/datasets/review_summary_dict.csv", index_col=0)
b = a.to_dict()
b

{'0': {"SBL Silk' n Stay Aloevera Cream 100g each [Pack of 2 ] [100gX2 , 200 g]\xa0\xa0(200 g)": '{\'summary_text\': "Oily., Nyce, very good, Very good, Osum cream bt not for whiting, Baje cream don\'t waist Manny, Amazing, Nice, Good product, It\'s a nice, Nice products."}',
  "LORD'S CREAM FOR COMPLEXION PACK OF 2 (EACH OF 25 GM )\xa0\xa0(50 g)": "{'summary_text': 'Good, Thanks, Ok, Good product, Good, Nice, Worst product, Super product must buy this..., Very nice, good product, Low quality product...., Very good ptoduct.'}",
  'Deyga Aloe Vera Gel 100% Pure Multipurpose for Face, Skin & Hair\xa0\xa0(100 g)': "{'summary_text': 'Good product, Very effective, Using it .. I like it, Nice, Good, The best aloevera gel i ever used., Nice, Alovera is pure But price is very high Please deyga make it affordable.'}",
  'Shahnaz Husain SHAMOIST 50GM\xa0\xa0(50 ml)': "{'summary_text': 'Good but darker in face after 1 week use, good, good product. Using for last 25years, Good mast, Very bad produ